In [1]:
# !wget https://github.com/udacity/deep-learning/blob/master/tensorboard/anna.txt
# enwik8: http://prize.hutter1.net/

In [2]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
import pdb
import time
import string

In [3]:
from pathlib import Path
path = Path("enwik8")
text = path.open(encoding="utf8").read()

In [4]:
printable = set(string.printable)
text_clean = list(filter(lambda x: x in printable, text))

# get the set of all characters
characters = tuple(set(text_clean))

# use enumeration to give the characters integer values
int2char = dict(enumerate(characters))

# create the look up dictionary from characters to the assigned integers
char2int = {char: index for index, char in int2char.items()}

# encode the text, using the character to integer dictionary
encoded = np.array([char2int[char] for char in text_clean])

In [7]:
to_gpu = True
def gpu(m):
    if to_gpu:
        return m.cuda()
    return m

In [8]:
def get_batches(arr, n_seqs_in_a_batch, n_characters):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    
    batch_size = n_seqs_in_a_batch * n_characters
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs_in_a_batch, -1))
    
    for n in range(0, arr.shape[1], n_characters):
        # The features
        x = arr[:, n:n+n_characters]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_characters]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [9]:
def ints_to_tensor(ints):
    return gpu(torch.tensor(ints).long().transpose(1, 0))

def xy_to_tensor(x, y):
    x = ints_to_tensor(x)
    y = torch.tensor(y.T).long()
    return x, gpu(y)

In [11]:
# build the model using the pytorch nn module
class CharLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_dim, batch_size, embedding_dim):
        super(CharLSTM, self).__init__()
        
        # init the meta parameters
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
        self.emb = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm_1 = nn.LSTMCell(input_size=embedding_dim, hidden_size=hidden_dim)
        self.lstm_2 = nn.LSTMCell(input_size=hidden_dim, hidden_size=hidden_dim) 
        
        self.dropout = nn.Dropout(p=0.5)

        # fully connected layer to connect the output of the LSTM cell to the output
        self.fc = nn.Linear(in_features=hidden_dim, out_features=vocab_size)
        
    def forward(self, x, hc, return_hc=False):
        seq_len = x.shape[0]
        batch_size = x.shape[1]
        
        # empty tensor for the output of the lstm
        output_seq = torch.empty((seq_len, batch_size, self.vocab_size))
        output_seq = gpu(output_seq)
        hc1, hc2 = hc, hc

        # for every step in the sequence
        for t in range(seq_len):
            out_t, hc1, hc2 = self.feed_one_x_t(x[t], hc1, hc2)
            output_seq[t] = out_t
        
        if return_hc:
            return output_seq, hc1, hc2
        return output_seq
            
    def init_hidden(self, bs=None):
        if bs is None:
            bs = self.batch_size
        # initialize the <hidden state> and the <cell state> to zeros
        return (gpu(torch.zeros(bs, self.hidden_dim)), gpu(torch.zeros(bs, self.hidden_dim)))
    
    def feed_one_x_t(self, x_t, hc1, hc2):
        # convert batch of single ints to batch of embeddings
        xt_emb = self.emb(x_t) # returns (batch_size, embedding_dim)

        # get the hidden and cell states from the first layer cell
        hc1 = self.lstm_1(xt_emb, hc1)
        h1, c1 = hc1 # unpack the hidden and the cell states from the first layer

        # pass the hidden state from the first layer to the cell in the second layer
        hc2 = self.lstm_2(h1, hc2)
        h2, c2 = hc2 # unpack the hidden and cell states from the second layer cell

        # form the output of the fc
        out_t = self.fc(self.dropout(h2))
        
        return out_t, hc1, hc2
    
    def feed_one_char(self, char, hc1, hc2):
        ints = [char2int[char]] # sequence of ints 
        ints = [ints] # a 1-batch of seqs
        x = ints_to_tensor(ints) # shape of (seq_len, batch_size)
        x_t = x[0] # take the first (single) part of the sequence
        
        return self.feed_one_x_t(x_t, hc1, hc2)
    
    def warm_up(self, base_str):
        hc = net.init_hidden(bs=1)
        ints = [char2int[c] for c in base_str]  # sequence of ints 
        ints = [ints] # a 1-batch of seqs
        x = ints_to_tensor(ints) # shape of (seq_len, batch_size)
        
        out, hc1, hc2 = self.forward(x, hc, return_hc=True)
        return out, hc1, hc2
    
    def sample_char(self, out_t, top_k=5):
        # apply the softmax to the output to get the probabilities of the characters
        out_t = F.softmax(out_t, dim=1)

        # out_t now holds the vector of predictions (1, vocab_size)
        # we want to sample 5 top characters
        p, top_char = out_t.topk(top_k) # returns tuple (top_values, top_indices)

        # get the top k characters by their probabilities
        top_char = top_char.cpu().squeeze().numpy()

        # sample a character using its probability
        p = p.detach().cpu().squeeze().numpy()
        char_int = np.random.choice(top_char, p = p/p.sum())
        
        return int2char[char_int]
        
    def predict(self, base_str, top_k=5, seq_len=128):
        self.eval()

        res = np.empty(seq_len+len(base_str), dtype="object")
        for i, c in enumerate(base_str):
            res[i] = c
        
        out_warm, hc1, hc2 = self.warm_up(base_str)
        out_t = out_warm[-1]

        for i in range(seq_len):
            char = self.sample_char(out_t, top_k)
            out_t, hc1, hc2 = self.feed_one_char(char, hc1, hc2)
            res[i + len(base_str)] = char
        
        return ''.join(res)
        

In [12]:
BS = 500 # 500
embedding_dim = 100
vocab_size=len(char2int)
hidden_dim = 512 # 512
seq_len = 128 # 128
seq_len_BS = seq_len * BS

In [13]:
print(f'BS: {BS}')
print(f'embedding_dim: {embedding_dim}')
print(f'vocab_size: {vocab_size}')
print(f'hidden_dim: {hidden_dim}')
print(f'seq_len: {seq_len}')
print(f'seq_len_BS: {seq_len_BS}')

BS: 500
embedding_dim: 100
vocab_size: 97
hidden_dim: 512
seq_len: 128
seq_len_BS: 64000


In [14]:
# get the validation and the training data
val_idx = int(len(encoded) * (1 - 0.1))
data, val_data = encoded[:val_idx], encoded[val_idx:]

In [15]:
# compile the network - sequence_len, vocab_size, hidden_dim, batch_size
net = CharLSTM(vocab_size=len(char2int), hidden_dim=hidden_dim, batch_size=BS, embedding_dim=embedding_dim)
net = gpu(net)

# define the loss and the optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# quick sanity

In [16]:
x, y = next(get_batches(data, BS, seq_len))

In [17]:
x, y = xy_to_tensor(x, y)

In [18]:
x.shape, y.shape

(torch.Size([128, 500]), torch.Size([128, 500]))

In [19]:
hc = net.init_hidden()
out = net(x, hc)
out.shape # (seq_len, batch_size, vocab_size)

torch.Size([128, 500, 97])

In [20]:
net.sample_char(out[-1, 0:1])

'o'

In [21]:
net.predict("The")

'TheLLJ(Fyfmok(mFof(mmmoEofmkVEEEmfpp>EE/fkpkkkA(fkkq(Lf(ff%kk=qqEEEEmqEmyEEkAmp2ymmEmypmLLLfLykLfkkqLLLLxyyfLpykf(kkf(%mfEEmpfkmEEp'

# Train

In [22]:
val_losses = list() # empty list for the validation losses
net.eval()

for epoch in range(10):
    
    # reinit the hidden and cell states
    hc = net.init_hidden()
    
    for i, (x, y) in enumerate(get_batches(data, BS, seq_len)):
        x_train, y_true = xy_to_tensor(x, y)
        optimizer.zero_grad() # zero out the gradients
    
        # forward pass
        t0 = time.time()
        y_pred = net(x_train, hc)
        if i == 0:
            print(f'elapsed forward: {time.time() - t0}')
    
        # calculate the loss
        # we need to calculate the loss across all batches, so we have to flat the y_true tensor
        loss = criterion(y_pred.view(BS*seq_len, -1), y_true.view(BS*seq_len)) # .contiguous()?
        
        # calculate the gradients
        t0 = time.time()
        loss.backward()
        if i == 0:
            print(f'elapsed backward: {time.time() - t0}')
        
        # update the parameters of the model
        optimizer.step()
    
        print("Epoch: {}, Batch: {}, Train Loss: {:.6f}".format(epoch, i, loss.item()))

        # feedback every 10 batches
        with torch.no_grad():
            if i % 10 == 0: 
                net.eval()
                
                x, y = next(get_batches(val_data, BS, seq_len))
                x_val, y_val = xy_to_tensor(x, y)
                hc_val = net.init_hidden()
                y_pred = net(x_val, hc_val)
                loss_val = criterion(y_pred.view(BS*seq_len, -1), y_val.view(BS*seq_len)) # .contiguous()?
                
                net.train()
                print("Epoch: {}, Batch: {}, Train Loss: {:.6f}, Validation Loss: {:.6f}".format(epoch, i, loss.item(), loss_val.item()))

                sample = net.predict("The", seq_len=100)
                print(sample)
                print()


elapsed forward: 0.3210113048553467
elapsed backward: 0.32642340660095215
Epoch: 0, Batch: 0, Train Loss: 4.583667
Epoch: 0, Batch: 0, Train Loss: 4.583667, Validation Loss: 4.532392
ThemEEomprmyyymymmmmmpEmEpprgyroeoo fooffofomofrmemetrtmrryyoyfmpyoffppet rret he ero te ot rrhrrytorrt

Epoch: 0, Batch: 1, Train Loss: 4.532494
Epoch: 0, Batch: 2, Train Loss: 4.449418
Epoch: 0, Batch: 3, Train Loss: 4.190681
Epoch: 0, Batch: 4, Train Loss: 3.696977
Epoch: 0, Batch: 5, Train Loss: 3.738126
Epoch: 0, Batch: 6, Train Loss: 3.662930
Epoch: 0, Batch: 7, Train Loss: 3.658451
Epoch: 0, Batch: 8, Train Loss: 3.618943
Epoch: 0, Batch: 9, Train Loss: 3.578385
Epoch: 0, Batch: 10, Train Loss: 3.552013
Epoch: 0, Batch: 10, Train Loss: 3.552013, Validation Loss: 3.510085
Ther e aae   at e  an aa nat  t a  at a  t  at ean a tan a   eena n   teat at tae n atnetn  nant    e  

Epoch: 0, Batch: 11, Train Loss: 3.537705
Epoch: 0, Batch: 12, Train Loss: 3.536778
Epoch: 0, Batch: 13, Train Loss: 3.536417
E

Epoch: 0, Batch: 135, Train Loss: 2.225668
Epoch: 0, Batch: 136, Train Loss: 2.236437
Epoch: 0, Batch: 137, Train Loss: 2.249898
Epoch: 0, Batch: 138, Train Loss: 2.237553
Epoch: 0, Batch: 139, Train Loss: 2.226410
Epoch: 0, Batch: 140, Train Loss: 2.199039
Epoch: 0, Batch: 140, Train Loss: 2.199039, Validation Loss: 2.162803
The to to coment in the [[Colestamaty]], in [[Charniss]] if to the chite of the stont crondaring and th

Epoch: 0, Batch: 141, Train Loss: 2.192783
Epoch: 0, Batch: 142, Train Loss: 2.190289
Epoch: 0, Batch: 143, Train Loss: 2.189692
Epoch: 0, Batch: 144, Train Loss: 2.185420
Epoch: 0, Batch: 145, Train Loss: 2.188447
Epoch: 0, Batch: 146, Train Loss: 2.175414
Epoch: 0, Batch: 147, Train Loss: 2.180696
Epoch: 0, Batch: 148, Train Loss: 2.159325
Epoch: 0, Batch: 149, Train Loss: 2.179427
Epoch: 0, Batch: 150, Train Loss: 2.141873
Epoch: 0, Batch: 150, Train Loss: 2.141873, Validation Loss: 2.120127
The stope one of [[Corso angie alcalita Andine]], [[Cared anstoustr

The [[Mirdory alding in Counse]] in [[Catian and are and tertications overing these and the also the su

Epoch: 0, Batch: 271, Train Loss: 1.871573
Epoch: 0, Batch: 272, Train Loss: 1.863790
Epoch: 0, Batch: 273, Train Loss: 1.864587
Epoch: 0, Batch: 274, Train Loss: 1.869111
Epoch: 0, Batch: 275, Train Loss: 1.836743
Epoch: 0, Batch: 276, Train Loss: 1.849661
Epoch: 0, Batch: 277, Train Loss: 1.862458
Epoch: 0, Batch: 278, Train Loss: 1.843054
Epoch: 0, Batch: 279, Train Loss: 1.857813
Epoch: 0, Batch: 280, Train Loss: 1.826992
Epoch: 0, Batch: 280, Train Loss: 1.826992, Validation Loss: 1.802058
The are ontinessive some and as the [[Brentaning]], in his also computer interved the framery the [[Ame

Epoch: 0, Batch: 281, Train Loss: 1.855003
Epoch: 0, Batch: 282, Train Loss: 1.863006
Epoch: 0, Batch: 283, Train Loss: 1.856808
Epoch: 0, Batch: 284, Train Loss: 1.851184
Epoch: 0, Batch: 285, Train Loss: 1.852801
Epoch: 0, Batch: 286, Train Loss: 1.841465
Epoch: 0, Batch: 287, Train Loss

Epoch: 0, Batch: 407, Train Loss: 1.719267
Epoch: 0, Batch: 408, Train Loss: 1.717492
Epoch: 0, Batch: 409, Train Loss: 1.715440
Epoch: 0, Batch: 410, Train Loss: 1.727224
Epoch: 0, Batch: 410, Train Loss: 1.727224, Validation Loss: 1.648085
The [[Sovio Cirity and Charanche]], terperite in the [[Ander Strouth]] and [[Morelling]]

= See [[Comma

Epoch: 0, Batch: 411, Train Loss: 1.740094
Epoch: 0, Batch: 412, Train Loss: 1.713094
Epoch: 0, Batch: 413, Train Loss: 1.716528
Epoch: 0, Batch: 414, Train Loss: 1.725482
Epoch: 0, Batch: 415, Train Loss: 1.701125
Epoch: 0, Batch: 416, Train Loss: 1.708837
Epoch: 0, Batch: 417, Train Loss: 1.707830
Epoch: 0, Batch: 418, Train Loss: 1.724313
Epoch: 0, Batch: 419, Train Loss: 1.712538
Epoch: 0, Batch: 420, Train Loss: 1.690246
Epoch: 0, Batch: 420, Train Loss: 1.690246, Validation Loss: 1.638817
The [[Charlen]] and the [[Servia]], [[1990]], [[Colland Centularia Corter|Streen Pritaris]], [[Barline]

Epoch: 0, Batch: 421, Train Loss: 1.694981
Epoch

Epoch: 0, Batch: 541, Train Loss: 1.579258
Epoch: 0, Batch: 542, Train Loss: 1.596283
Epoch: 0, Batch: 543, Train Loss: 1.595293
Epoch: 0, Batch: 544, Train Loss: 1.603595
Epoch: 0, Batch: 545, Train Loss: 1.604236
Epoch: 0, Batch: 546, Train Loss: 1.592978
Epoch: 0, Batch: 547, Train Loss: 1.605393
Epoch: 0, Batch: 548, Train Loss: 1.617438
Epoch: 0, Batch: 549, Train Loss: 1.621979
Epoch: 0, Batch: 550, Train Loss: 1.623667
Epoch: 0, Batch: 550, Train Loss: 1.623667, Validation Loss: 1.555794
The [[Charle Artic Broth]], a toth all are the certain that ways are the car of this continoons and pre

Epoch: 0, Batch: 551, Train Loss: 1.606227
Epoch: 0, Batch: 552, Train Loss: 1.604063
Epoch: 0, Batch: 553, Train Loss: 1.594911
Epoch: 0, Batch: 554, Train Loss: 1.590199
Epoch: 0, Batch: 555, Train Loss: 1.609511
Epoch: 0, Batch: 556, Train Loss: 1.605970
Epoch: 0, Batch: 557, Train Loss: 1.611484
Epoch: 0, Batch: 558, Train Loss: 1.592980
Epoch: 0, Batch: 559, Train Loss: 1.618293
Epoch: 0

...

The day to a period, and was the context on a series with the propheties are a similar to the property 

Epoch: 0, Batch: 1351, Train Loss: 1.413596
Epoch: 0, Batch: 1352, Train Loss: 1.404449
Epoch: 0, Batch: 1353, Train Loss: 1.425090
Epoch: 0, Batch: 1354, Train Loss: 1.414446
Epoch: 0, Batch: 1355, Train Loss: 1.415369
Epoch: 0, Batch: 1356, Train Loss: 1.409480
Epoch: 0, Batch: 1357, Train Loss: 1.405063
Epoch: 0, Batch: 1358, Train Loss: 1.410306
Epoch: 0, Batch: 1359, Train Loss: 1.406739
Epoch: 0, Batch: 1360, Train Loss: 1.397062
Epoch: 0, Batch: 1360, Train Loss: 1.397062, Validation Loss: 1.373396
The series of technology are also allowing a choose are still asserts to the [[motor]] and a concernanc

Epoch: 0, Batch: 1361, Train Loss: 1.408603
Epoch: 0, Batch: 1362, Train Loss: 1.392436
Epoch: 0, Batch: 1363, Train Loss: 1.399198
Epoch: 0, Batch: 1364, Train Loss: 1.403230
Epoch: 0, Batch: 1365, Train Loss: 1.398050
Epoch: 0, Batch: 1366, Train Loss: 1.381221
Epoch: 0, Batch

Epoch: 1, Batch: 90, Train Loss: 1.370163
Epoch: 1, Batch: 90, Train Loss: 1.370163, Validation Loss: 1.363054
The [[Andr Steeller Sena Sox Stock]], and [[California]] and are two countries than office a power and 

Epoch: 1, Batch: 91, Train Loss: 1.392171
Epoch: 1, Batch: 92, Train Loss: 1.380057
Epoch: 1, Batch: 93, Train Loss: 1.376597
Epoch: 1, Batch: 94, Train Loss: 1.357566
Epoch: 1, Batch: 95, Train Loss: 1.387971
Epoch: 1, Batch: 96, Train Loss: 1.361760
Epoch: 1, Batch: 97, Train Loss: 1.346769
Epoch: 1, Batch: 98, Train Loss: 1.361317
Epoch: 1, Batch: 99, Train Loss: 1.377467
Epoch: 1, Batch: 100, Train Loss: 1.378004
Epoch: 1, Batch: 100, Train Loss: 1.378004, Validation Loss: 1.363886
The [[Protestantist]] [[compressive section|producer]].

The [[President]] in the [[1987s]], when the p

Epoch: 1, Batch: 101, Train Loss: 1.380582
Epoch: 1, Batch: 102, Train Loss: 1.379062
Epoch: 1, Batch: 103, Train Loss: 1.385159
Epoch: 1, Batch: 104, Train Loss: 1.380651
Epoch: 1, Batch:

Epoch: 1, Batch: 224, Train Loss: 1.381423
Epoch: 1, Batch: 225, Train Loss: 1.378621
Epoch: 1, Batch: 226, Train Loss: 1.382305
Epoch: 1, Batch: 227, Train Loss: 1.378611
Epoch: 1, Batch: 228, Train Loss: 1.364095
Epoch: 1, Batch: 229, Train Loss: 1.371183
Epoch: 1, Batch: 230, Train Loss: 1.371686
Epoch: 1, Batch: 230, Train Loss: 1.371686, Validation Loss: 1.350687
The [[Media]], the set of an information, and with the fighting community on the [[Persian Estonian liv

Epoch: 1, Batch: 231, Train Loss: 1.368854
Epoch: 1, Batch: 232, Train Loss: 1.359873
Epoch: 1, Batch: 233, Train Loss: 1.364185
Epoch: 1, Batch: 234, Train Loss: 1.361229
Epoch: 1, Batch: 235, Train Loss: 1.370606
Epoch: 1, Batch: 236, Train Loss: 1.370470
Epoch: 1, Batch: 237, Train Loss: 1.347319
Epoch: 1, Batch: 238, Train Loss: 1.372731
Epoch: 1, Batch: 239, Train Loss: 1.392890
Epoch: 1, Batch: 240, Train Loss: 1.363231
Epoch: 1, Batch: 240, Train Loss: 1.363231, Validation Loss: 1.350721
The [[Automatic Arthura]

Epoch: 1, Batch: 360, Train Loss: 1.329503, Validation Loss: 1.340624
The [[Bruce Catalonia]] and [[Serbia Same Basel]]s.  

In [[1641]], he could be changed on their strip 

Epoch: 1, Batch: 361, Train Loss: 1.361532
Epoch: 1, Batch: 362, Train Loss: 1.357049
Epoch: 1, Batch: 363, Train Loss: 1.349352
Epoch: 1, Batch: 364, Train Loss: 1.367308
Epoch: 1, Batch: 365, Train Loss: 1.357359
Epoch: 1, Batch: 366, Train Loss: 1.355057
Epoch: 1, Batch: 367, Train Loss: 1.348690
Epoch: 1, Batch: 368, Train Loss: 1.353063
Epoch: 1, Batch: 369, Train Loss: 1.371551
Epoch: 1, Batch: 370, Train Loss: 1.369453
Epoch: 1, Batch: 370, Train Loss: 1.369453, Validation Loss: 1.338872
The [[Canada]] and [[Meral Source]] with the sons is the [[American Control of California]], third indi

Epoch: 1, Batch: 371, Train Loss: 1.388599
Epoch: 1, Batch: 372, Train Loss: 1.383344
Epoch: 1, Batch: 373, Train Loss: 1.353538
Epoch: 1, Batch: 374, Train Loss: 1.354966
Epoch: 1, Batch: 375, Train Loss: 1.366530
Epoch

KeyboardInterrupt: 

In [ ]:
net.predict("God i")